<a href="https://colab.research.google.com/github/JAIDHEER007/Modular-Times-Table/blob/main/TTM_Vedio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
from math import sin, cos, sqrt, radians
from IPython.display import display
import urllib.request

import os
from os.path import join
import datetime

import os
import cv2
import re

In [6]:
def createFolder(divisions, k) -> str:
    # Current Working Directory
    pdir = os.getcwd()
    
    #Getting Current Time
    curr = datetime.datetime.now()
    dateStr = str(curr.date())
    timeStr = str(curr.time())

    #Folder name
    directory = "RUN_" + dateStr.replace('-','_') + "_" + timeStr.replace(':', '_')
    
    #Final Path
    fpath = join(pdir, directory)
    
    #Creating the directory
    os.mkdir(fpath)
    
    # Creating the Images SubFolder
    imgPath = join(fpath, 'Images')
    os.mkdir(imgPath)
    
    # Writing usefull files
    info = """
           author: S.Jaidheer
           github: JAIDHEER007
           
           Implementing Modular Times Tables
           
           INFO
           =====================================================
           Date: {fdate}
           time: {ftime}
            
           Divisions: {fdivs}
           K: {kval}
           
           Images will be saved in Images Folder
           Once the program ends, Execute makeVideo.py program
           """.format(fdate = dateStr, ftime = timeStr, fdivs = divisions, kval = k)
           
    with open(join(fpath, 'info.txt'), 'w') as fileHandle:
        fileHandle.writelines(info)
           
    return fpath


In [7]:
class timeTableM:
    def __init__(self, size, divisions, k, lwidth = 1, cwidth = 2):
        self.size = size
        self.divisions = divisions
        self.k = k
        self.radius = sqrt((self.size[0] ** 2) + (self.size[1] ** 2)) // 3.5
        self.theta = 360 / self.divisions
        self.__lwidth = lwidth
        self.__cwidth = cwidth

        self._fpath = createFolder(self.divisions, self.k)
        print(self._fpath)
        self._imgPath = join(self._fpath, 'Images')

        self.__coordinates = []
        shiftX = -(self.size[0] / 2)
        shiftY = self.size[1] / 2

        for i in range(self.divisions):
            rad = radians(i * self.theta)
            self.__coordinates.append(((self.radius * cos(rad)) - shiftX, shiftY - (self.radius * sin(rad))))

        self.__baseImage = Image.new(mode = 'RGB', size = size)
        draw = ImageDraw.Draw(im = self.__baseImage)
        center = (self.size[0] // 2, self.size[1] // 2)
        leftCorner = (center[0] - self.radius, center[1] - self.radius)
        rightCorner = (center[0] + self.radius, center[1] + self.radius)

        boundingBox = [leftCorner, rightCorner]
        draw.ellipse(xy = boundingBox, fill = None, outline = (255, 255, 255), width = self.__cwidth)

        fontUrl = 'https://www.1001fonts.com/download/font/open-sans.regular.ttf'
        # fontUrl = 'https://www.1001fonts.com/download/font/open-sans.italic.ttf'

        urllib.request.urlretrieve(fontUrl, 'font.ttf')

        # Manual Editing
        # Plans to automate it
        self.__textX = 100
        self.__textY = self.size[1] - 100
        self.__fontSize = 50

    def showBaseImage(self):
        display(self.__baseImage)

    def showBaseImageText(self):
        img = self.__baseImage.copy()
        draw = ImageDraw.Draw(im = img)
        customFont = ImageFont.truetype('/content/font.ttf', self.__fontSize)
        text = "Divisions: 553  K: 007"
        draw.text(xy = (self.__textX, self.__textY),font = customFont, text = text, fill = (255, 255, 255))
        display(img)


    def start(self):
        img = self.__baseImage.copy()
        draw = ImageDraw.Draw(im = img)
        customFont = ImageFont.truetype('font.ttf', self.__fontSize)
        text = "Divisions: {divs}  Division: {div}  K: {kval}"
        for i in range(self.divisions):
            draw.line(xy = [self.__coordinates[i], self.__coordinates[(i * self.k) % self.divisions]], fill = (0, 0, 255), width = self.__lwidth)
            
            textImg = img.copy()
            textDraw = ImageDraw.Draw(im = textImg)
            textDraw.text(xy = (self.__textX, self.__textY),font = customFont, text = text.format(divs = self.divisions, div = i, kval = self.k), fill = (255, 255, 255))

            textImg.save(join(self._imgPath, 'testImg_{div}.png'.format(div = i)))
    
    def saveVedio(self):
        video_name = join(self._fpath, "Vedio_{divs}_{kval}.mp4".format(divs = self.divisions, kval = self.k))

        images = [img for img in os.listdir(self._imgPath) if img.endswith(".png")]

        pattern = re.compile(r'[0-9]+')
        images.sort(key = lambda image: int(pattern.findall(image)[-1]))

        frame = cv2.imread(os.path.join(self._imgPath, images[0]))

        height, width = frame.shape[:2]

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video = cv2.VideoWriter(video_name, fourcc, 35, (width, height)) 

        for image in images:
            video.write(cv2.imread(os.path.join(self._imgPath, image)))

        cv2.destroyAllWindows() 
        video.release()
    

In [8]:
divisions = 200
k = 2
ttm = timeTableM(size = (2000, 2000), divisions = divisions, k = k, cwidth = 3)

/content/RUN_2022_01_06_08_24_59.488086


In [9]:
ttm.start()
ttm.saveVedio()